In [36]:
import pandas as pd
from datetime import datetime


# Crawl data from yahoo finance

In [37]:


def getCrumb(ticker):
    URL  = "https://finance.yahoo.com/quote/%s/history" % ticker
    import requests 
    import re
    r = requests.get(url = URL) 
    content = str(r.content)
    r1 = re.findall(r"CrumbStore\":\{\"crumb\":\"[A-Za-z0-9.\\\-]+\"",content)[0]
    crumb = r1.replace("CrumbStore\":{\"crumb\":\"","").replace('"',"")
    return crumb,r.cookies

def get_data(data_type, symbol, start_date, end_date):
    import requests 
    import re
    from io import StringIO
    api_url = "https://query1.finance.yahoo.com/v7/finance/download/{0}?period1={1}&period2={2}&interval={3}&events={4}&crumb={5}"
    (crumb, cookies)=getCrumb(symbol)
    api_url = api_url.format(symbol,start_date,end_date,"1d", data_type, crumb)
    r = requests.get(url = api_url, cookies = cookies.get_dict(".yahoo.com"))  
    content = str(r.content).replace(",",'","').replace("\\n",'"\r\n"').replace("b\'","").replace("'","")
    res = '"'+content+'"'
    df = pd.read_csv(StringIO(res), sep=",",engine='python')
    return df

def get_price_data(stock, start_date, end_date):
    print("Getting price data ...")
    price = get_data("history", stock, start_date, end_date)
    price.rename(columns={'Date':'date','Open':'open','High':'high',
                          'Low':'low','Close':'close','Adj Close':'adjClose','Volume':'vol'}, inplace=True)
    price["date"]=pd.to_datetime(price['date'])
    price = price.set_index(["date"])
    return price
def get_dividend_data(stock, start_date, end_date):
    print("Getting dividend data ...")
    dividend = get_data("div", stock, start_date, end_date)
    dividend.rename(columns={'Date':'date','Dividends':'dividend'}, inplace=True)
    dividend["date"]=pd.to_datetime(dividend['date'])
    dividend = dividend.set_index(["date"])
    return dividend

# Calculate returns

In [111]:

def calculate_profit(price, dividend, init_amt = 10000, fee = 0, monthly_topup = 0):
    print("Calculating returns ...")
    from datetime import timedelta, date
    data = pd.DataFrame([]) 
    init_close = -1
    total_spend = init_amt
    amount = init_amt
    shares = 0
    curDate = price.index.min()
    last_close = 0
    last_open = 0
    pre_date = None
    while True:
        curDiv = 0
        if(curDate in price.index):
            if(pre_date is not None and pre_date.month != curDate.month):
                amount = amount + monthly_topup
                total_spend = total_spend + monthly_topup
            pre_date = curDate
            curRec = price.loc[[curDate]]
            curOpen = float(curRec["open"].values[0])
            curClose = float(curRec["close"].values[0])
            last_open = curOpen
            last_close = curClose
            if(init_close==-1):
                init_close = last_close
            if(amount>0):
                #buy shares
                share_can_buy = (int)(amount/(curOpen*(1+fee)))
                amount = amount - share_can_buy*(curOpen*(1+fee))
                shares = shares + share_can_buy
        if(curDate in dividend.index):
            curDiv = dividend.loc[[curDate]]["dividend"].values[0]
            amount = amount + (shares*float(curDiv))
        if(curDate == price.index.max()):
            break
        gross = shares*curClose + amount
        data = data.append({'Date' : curDate ,
                            'Spent': round(total_spend,2),
                            'Gross' : round(gross,2),
                            'Dividend': curDiv,
                            'Close': curClose,
                            'ClosePct': round(((curClose-init_close)/init_close)*100,2),
                            'NetPct' :  round(((gross-total_spend)/total_spend)*100,2),
                           } , ignore_index=True)
        curDate = curDate + timedelta(days=1)
    return data

# Plot line chart

In [130]:
def plot_chart(data, height, width, cols = []):
    print("Ploting chart ...")
    import plotly.graph_objs as go 
    from datetime import datetime
    from ipywidgets import interact, interactive, fixed, interact_manual
    import ipywidgets as widgets
    from plotly.subplots import make_subplots
    
    df = data.set_index('Date')
    df = df.sort_index(ascending=True)
    fig = go.FigureWidget(make_subplots(rows=1, specs=[[{"secondary_y": True}]]))
    
    
    for col in cols:
        if(col == "Dividend"):
            fig.add_trace(
                go.Scattergl(x=list(df[df["Dividend"]>0].index), y=list(df[df["Dividend"]>0][col]),
                name = col, marker=dict(color="green", size=2), line = dict(color='grey', width=1, dash='dash')),
                row=1, col=1,
                secondary_y=True
            )
        else:
            fig.add_trace(
                go.Scattergl(x=list(df.index), y=list(df[col]),
                name = col),
                row=1, col=1
            )
            
    
    df["const"] = 0
    
    fig.add_trace(
            go.Scattergl(x=list(df.index), y=list(df["const"]), line = dict(color='grey', width=1, dash='dash'),
            name = "zero"),
            row=1, col=1
    )
    fig.layout.xaxis=dict(
            anchor='x',
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label='1m',
                         step='month',
                         stepmode='backward'),
                    dict(count=6,
                         label='6m',
                         step='month',
                         stepmode='backward'),
                    dict(count=1,
                        label='YTD',
                        step='year',
                        stepmode='todate'),
                    dict(count=1,
                        label='1y',
                        step='year',
                        stepmode='backward'),
                    dict(step='all')
                ])
            ),
            type='date'
        )
    fig['layout'].update(height=height, width=width) 
    return fig

In [133]:
start_date = datetime(2017,4,1).strftime('%s')
end_date = datetime(2020,4,1).strftime('%s')
stock = "G3B.SI"
init_amt = 10000
fee = 0.01  #1%
monthly_topup = 0
price_data = get_price_data(stock, start_date, end_date)
dividend_data = get_dividend_data(stock, start_date, end_date)
returns = calculate_profit(price_data, dividend_data, init_amt, fee, monthly_topup)

Getting price data ...
Getting dividend data ...
Calculating returns ...


In [134]:

plot_chart(returns, 500, 900, ['Spent', 'Gross',  'Dividend', 'Close', 'ClosePct','NetPct'])

Ploting chart ...


FigureWidget({
    'data': [{'name': 'Spent',
              'type': 'scattergl',
              'uid': 'e38f158…